In [1]:
## load all librabies and trained model
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import numpy as np
import pandas as pd

2025-11-11 23:17:25.172886: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-11 23:17:25.194734: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
## load the ann trainned model ,scaler pikle on ehot
model=load_model('model.h5')

## load the encoder and scaler
with open('label_encoder_gender.pkl','rb') as file:
    label_encoder_gender=pickle.load(file)
    
with open('onehot_encoder_geo.pkl','rb') as file:
    onehot_encoder_geo=pickle.load(file)
    
with open('scaler.pkl','rb') as file:
    scaler=pickle.load(file)

In [3]:
## Example input data for prediction
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [4]:
## one-hot encode 'Geography'
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/home/abc/Desktop/ANN/annenv/lib/python3.11/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])

In [6]:
## combine one-hot encoded columns with input data
input_data=pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_data
## now this row will go for prediction

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [7]:
## Encode Categorical varaible
input_df['Gender']=label_encoder_gender.transform(input_df['Gender'])

In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [9]:
## 
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [10]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [11]:
## sacling the input data
input_scaled=scaler.transform(input_df)

In [12]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [13]:
## prediction
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step


array([[0.04241772]], dtype=float32)

In [14]:
prediction_proba=prediction[0][0]

In [15]:
prediction_proba

0.04241772

In [16]:
if prediction_proba >0.5:
    print("The customer is likely to churn")
    
else:
    print("The  customer is not likely to churn")

The  customer is not likely to churn
